In [1]:
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
import os

import sys
sys.path.insert(0, '/Users/guilhermeosorio/Desktop/ARGO/argo-backend')

from app.utils.langchain import DocPath, generate_text_from_path, generate_chunks
from app.config import app_config
from app.utils.embeddings import EmbeddingRequest, UpsertRequest, generate_embedding, upsert_embedding
from app.utils.pinecone import pinecone_client
from app.models.vectordb import Vector, VectorMetaData

from dotenv import load_dotenv
load_dotenv()

CLAUDE_API_KEY = os.getenv('CLAUDE_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_INDEX_HOST = os.getenv('PINECONE_INDEX_HOST')

/Users/guilhermeosorio/Library/Caches/pypoetry/virtualenvs/argo-backend-UDiPBMrR-py3.12/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
directory = '../../../_AI Input Proposals'

def list_rfp_files(directory):
    rfp_files = []
    for dirpath, _, filenames in os.walk(directory):
        for filename in filenames:
            if filename == "RFP.pdf":
                rfp_files.append(os.path.join(dirpath, filename))
    return rfp_files

directory += '/Known'
rfp_paths = list_rfp_files(directory)
rfp_paths


['../../../_AI Input Proposals/Known/VFPA Gateway/RFP.pdf',
 '../../../_AI Input Proposals/Known/Cullen Commission/RFP.pdf',
 '../../../_AI Input Proposals/Known/Canada Energy Regulator/RFP.pdf',
 '../../../_AI Input Proposals/Known/City of Castlegar/RFP.pdf',
 '../../../_AI Input Proposals/Known/Broadway Subway Project/RFP.pdf',
 '../../../_AI Input Proposals/Known/BC Housing Strat Com/RFP.pdf',
 '../../../_AI Input Proposals/Known/Metro Vancouver/RFP.pdf',
 '../../../_AI Input Proposals/Known/CoV - Creative Design/RFP.pdf',
 '../../../_AI Input Proposals/Known/DWV/RFP.pdf',
 '../../../_AI Input Proposals/Known/Cowichan Tribes/RFP.pdf',
 '../../../_AI Input Proposals/Known/BC Transit/RFP.pdf',
 '../../../_AI Input Proposals/Known/RDN/RFP.pdf']

In [3]:
test_rfp_path = rfp_paths[0] # VFPA Gateway
# pdfs to chunks
input = DocPath(doc_path=test_rfp_path)
text = generate_text_from_path(request=input)
chunks = generate_chunks(request=text).texts
# chunks to pinecone
rfp_name = test_rfp_path.split('/')[-2]

In [4]:
test_rfp = "VFPA Gateway"

In [5]:
llm_small = ChatAnthropic(temperature=0, anthropic_api_key=CLAUDE_API_KEY, model="claude-3-haiku-20240307") # smallest
llm_medium = ChatAnthropic(temperature=0, anthropic_api_key=CLAUDE_API_KEY, model="claude-3-sonnet-20240229") # medium
llm_large = ChatAnthropic(temperature=0, anthropic_api_key=CLAUDE_API_KEY, model="claude-3-opus-20240229") # larger

In [6]:
summarized_chunks = []

for chunk in chunks:
    system = """
    You are an expert consultant analyzing an RFP that was just submitted for your review.
    You will be given a chunk taken out of this RFP document. Your goal is to summarize the given chunk, keeping the most relevant parts of the content.
    When summarizing the RFP chunk, start directly with the summary content, without mentioning 'the given chunk summarizes' or any similar phrasing.
    If any instruction(s) about the response to be submitted to the RFP are present in the chunk, it is crucial that you keep them in your summarization.
    """
    human = """
    Chunk:
    {chunk}

    Summarized chunk:
    """

    RFP_summarizer = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

    chain = RFP_summarizer | llm_small

    summarized_chunk = chain.invoke(
        {
            "chunk": chunk
        }
    )
    summarized_chunks.append(summarized_chunk.content)

In [7]:
for i, r in enumerate(chunks):
    print(f"CHUNK {i}")
    print(f"{r}\n")

CHUNK 0
VANCOUVER FRASER PORT AUTHORITY  

REQUEST FOR PROPOSAL: #P210205-01 

Graphic design support for Gateway projects 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
1.0 

NOTICE TO PROPONENTS 

THIS IS AN INVITATION FOR PROPOSALS AND NOT A TENDER CALL. 

(A.) 

(B.) 

Proposals are to be submitted electronically via email (one pdf file attachment preferred) to the 
attention of the contact addresses identified in Section 1.0 (C.) by the closing time of 3:00 p.m. 
local Vancouver time on February 5, 2021.  The subject line should include the Request for 
Proposal (“RFP”) number and title “P210205-01”.  It is the responsibility of the proponent to 
ensure that the emailed proposal is successfully sent and received by the closing time.  Late 
proposals may not be considered. 

The deadline for Questions relating to this RFP is by 5:00 p.m. on January 25, 2021 (“the 
Deadline”). The Deadline will apply to any addendum issued prior to the Deadline, unless a new 
Deadlin

In [8]:
for i, r in enumerate(summarized_chunks):
    print(f"CHUNK {i}")
    print(f"{r}\n")

CHUNK 0
The summary of the given chunk is as follows:

The Vancouver Fraser Port Authority is seeking a graphic designer or agency to assist in developing and executing public materials to support a suite of infrastructure projects the port authority is leading in 2021. The port authority is undertaking various projects to enhance the movement of trade-related cargo through Greater Vancouver, improve mobility and safety, protect the environment, and alleviate the impacts of growing trade on local communities. Since 2014, the port authority has been working collaboratively with the Gateway Transportation Collaboration Forum to identify projects that will improve the trade and transportation network to meet growth anticipated by 2030. The Greater Vancouver Gateway 2030 Strategy was created to identify smart infrastructure investments to ensure an efficient and competitive network to support Canadian trade, while also addressing the community impacts of anticipated trade and population gr

In [9]:
# Concatenate the strings into a single string
summarized_chunks_str = "\n".join(summarized_chunks)
print(summarized_chunks_str)

The summary of the given chunk is as follows:

The Vancouver Fraser Port Authority is seeking a graphic designer or agency to assist in developing and executing public materials to support a suite of infrastructure projects the port authority is leading in 2021. The port authority is undertaking various projects to enhance the movement of trade-related cargo through Greater Vancouver, improve mobility and safety, protect the environment, and alleviate the impacts of growing trade on local communities. Since 2014, the port authority has been working collaboratively with the Gateway Transportation Collaboration Forum to identify projects that will improve the trade and transportation network to meet growth anticipated by 2030. The Greater Vancouver Gateway 2030 Strategy was created to identify smart infrastructure investments to ensure an efficient and competitive network to support Canadian trade, while also addressing the community impacts of anticipated trade and population growth.

T

In [10]:
system = """
You are an expert consultant analyzing an RFP that was just submitted for your review.
Based on an RFP summary, create a bulleted list to serve as a structure for the response to the RFP.
Your main goal is to provide a bulleted point list to serve as a reference in the response, as the team is looking to automate the process of generating an initial draft to respond to RFPs.
You are part of a company called Kirk & Co., which is one of Canada's leading firms specializing in strategic communications, consultation and engagement.
In the response structure, separate the sections with the substring '\n\n'
"""
human = """
RFP Summary:
{rfp_summary}

Response structure:
"""

response_structurer = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = response_structurer | llm_medium

response_structure = chain.invoke(
    {
        "rfp_summary": summarized_chunks_str
    }
).content


In [11]:
print(response_structure)

Based on the RFP summary, here is a bulleted list structure for the response:

1. Introduction
   - Overview of Kirk & Co. and its capabilities
   - Understanding of the project requirements and objectives

2. Proposed Services and Approach
   - Detailed description of services offered
   - Methodology and approach for each project phase
   - Project management and communication plan
   - Quality assurance and control measures

3. Company Background and Experience
   - Company history and profile
   - Relevant experience with similar projects
   - Key personnel and their qualifications
   - Client references and testimonials

4. Sustainability and Social Responsibility
   - Environmental, ethical, and sustainable initiatives
   - Community involvement and Aboriginal participation
   - Commitment to VFPA's vision and values

5. Cost Proposal
   - Hourly fee structure for individual service providers
   - Estimated resource utilization and costs
   - Additional fees or disbursements, if 

In [12]:
response_structure

"Based on the RFP summary, here is a bulleted list structure for the response:\n\n1. Introduction\n   - Overview of Kirk & Co. and its capabilities\n   - Understanding of the project requirements and objectives\n\n2. Proposed Services and Approach\n   - Detailed description of services offered\n   - Methodology and approach for each project phase\n   - Project management and communication plan\n   - Quality assurance and control measures\n\n3. Company Background and Experience\n   - Company history and profile\n   - Relevant experience with similar projects\n   - Key personnel and their qualifications\n   - Client references and testimonials\n\n4. Sustainability and Social Responsibility\n   - Environmental, ethical, and sustainable initiatives\n   - Community involvement and Aboriginal participation\n   - Commitment to VFPA's vision and values\n\n5. Cost Proposal\n   - Hourly fee structure for individual service providers\n   - Estimated resource utilization and costs\n   - Additional

In [13]:
response_sections = response_structure.split("\n\n")
response_sections

['Based on the RFP summary, here is a bulleted list structure for the response:',
 '1. Introduction\n   - Overview of Kirk & Co. and its capabilities\n   - Understanding of the project requirements and objectives',
 '2. Proposed Services and Approach\n   - Detailed description of services offered\n   - Methodology and approach for each project phase\n   - Project management and communication plan\n   - Quality assurance and control measures',
 '3. Company Background and Experience\n   - Company history and profile\n   - Relevant experience with similar projects\n   - Key personnel and their qualifications\n   - Client references and testimonials',
 "4. Sustainability and Social Responsibility\n   - Environmental, ethical, and sustainable initiatives\n   - Community involvement and Aboriginal participation\n   - Commitment to VFPA's vision and values",
 "5. Cost Proposal\n   - Hourly fee structure for individual service providers\n   - Estimated resource utilization and costs\n   - Addi

In [14]:
system = """
You are an expert consultant working on drafting a response to an RFP.
Based on an RFP summary and a given outline for a section of the response, develop the section's content.
Your main goal is to provide the final draft for this section. After all sections have been generated, they will be concatenated in the final draft of the response.
You are part of a company called Kirk & Co., which is one of Canada's leading firms specializing in strategic communications, consultation and engagement.
"""
human = """
RFP Summary:
{rfp_summary}

Section outline:
{section_structure}

Section:
"""

response_composer = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = response_composer | llm_medium

final_response_sections = []

for section in response_sections:


    section_content = chain.invoke(
        {
            "rfp_summary": summarized_chunks_str,
            "section_structure": section
        }
    ).content
    final_response_sections.append(section_content)

In [15]:
for i, r in enumerate(final_response_sections):
    print(f"SECTION {i}")
    print(f"{r}\n")

SECTION 0
Based on the RFP summary and outline, here is a draft for the section:

Section: Proposed Services, Methodology and Approach

At Kirk & Co., we specialize in strategic communications, consultation, and engagement for major infrastructure projects. Our approach is centered on developing compelling visuals and messaging that resonate with diverse stakeholders while adhering to brand guidelines. We propose the following services and methodology:

Services:
- Graphic design for print and digital collateral (infographics, ads, presentations, etc.)
- Video production and animation 
- Copywriting and content development
- Print management and production coordination

Methodology:
1. Discovery & Planning
   - Review VFPA brand guidelines, style guides, communications strategies
   - Conduct project briefings with VFPA team 
   - Develop creative concepts and messaging frameworks

2. Design & Production 
   - Iterative design process with regular checkpoints
   - Copywriting, editing,

In [16]:
final_response_sections_str = "\n".join(final_response_sections)
print(final_response_sections_str)

Based on the RFP summary and outline, here is a draft for the section:

Section: Proposed Services, Methodology and Approach

At Kirk & Co., we specialize in strategic communications, consultation, and engagement for major infrastructure projects. Our approach is centered on developing compelling visuals and messaging that resonate with diverse stakeholders while adhering to brand guidelines. We propose the following services and methodology:

Services:
- Graphic design for print and digital collateral (infographics, ads, presentations, etc.)
- Video production and animation 
- Copywriting and content development
- Print management and production coordination

Methodology:
1. Discovery & Planning
   - Review VFPA brand guidelines, style guides, communications strategies
   - Conduct project briefings with VFPA team 
   - Develop creative concepts and messaging frameworks

2. Design & Production 
   - Iterative design process with regular checkpoints
   - Copywriting, editing, and proof

In [17]:
len(final_response_sections_str)

15690